In [37]:
import os
import glob
import pandas as pd
import shutil
import numpy as np
from tqdm import tqdm

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [40]:
checkplot_folder = '//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/face_motion_ac_mask/checkplots'
motmask_source_folder = '//allen/programs/braintv/workgroups/cortexmodels/peterl/facemap_results'
motmask_destination_folder = '//allen/programs/braintv/workgroups/nc-ophys/visual_behavior/qc_plots/face_motion_ac_mask/motmasks'

In [41]:
folders = ['_EDGE_CASES','_GOOD_MASKS','_BAD_MASKS']

In [44]:
_list = []
for folder in folders:
    
    for fn in glob.glob(os.path.join(checkplot_folder, folder, 'check_plot*')):
        osid = int(fn.split('plot_')[1].split('_video')[0])
        _list.append({'ophys_session_id': osid, 'folder_name': folder})

check_plot_df = pd.DataFrame(_list)
check_plot_df['motmask_exists'] = None

In [45]:
check_plot_df

,ophys_session_id,folder_name,motmask_exists
0,1066944927,_EDGE_CASES,None
1,1037717307,_EDGE_CASES,None
2,1010684546,_EDGE_CASES,None
3,1052482256,_EDGE_CASES,None
4,1039246790,_EDGE_CASES,None
...,...,...,...
787,847494416,_BAD_MASKS,None
788,754695045,_BAD_MASKS,None
789,918718249,_BAD_MASKS,None
790,965038745,_BAD_MASKS,None


In [48]:
check_plot_df['folder_name'].value_counts(normalize=True)

_GOOD_MASKS    0.584596
_BAD_MASKS     0.280303
_EDGE_CASES    0.135101
Name: folder_name, dtype: float64

In [49]:
for idx, row in tqdm(check_plot_df.iterrows(), total = len(check_plot_df)):
    osid = row['ophys_session_id']
    motmask_source_filename = os.path.join(motmask_source_folder, 'motMask_{}_video-0.png'.format(osid))
    try:
        shutil.copy(
            motmask_source_filename,
            os.path.join(motmask_destination_folder, row['folder_name'])
        )
        check_plot_df.at[idx, 'motmask_exists'] = True
    except FileNotFoundError:
        check_plot_df.at[idx, 'motmask_exists'] = False

100%|██████████| 792/792 [00:29<00:00, 26.43it/s]


In [52]:
check_plot_df.to_csv(os.path.join(checkplot_folder, '2021.06.30_check_plot_qc_summary.csv'), index=False)